In [1]:
import sys

In [2]:
""" 
seed must be a binary string 
returns the current state after computing output for output_length number of bits and the 
number of output bits as mentioned in output_length

Last output bit is the result of the first computation

"""
def lfsr(seed,
         taps,
         output_length):
    xor = 0
    state = seed
    counter = 0
    output = ''
    for i in range (0,output_length):
        xor = 0
        for t in taps:
            xor = (xor + int(state[t]))%2
        state = state[0:len(state)-1]
        state = str(xor) + str(state)
        output = str(xor) + output
    return state,output

In [3]:
""" Length of key = length of data. """
def one_time_pad(key,data):
    if len(key) != len(data):
        raise ValueError("Key and data should be of same length for one time pad.")
    encrypted_data = ''
    for i in range(0,len(key)):
        encrypted_data = encrypted_data+str((int(key[i]) + int(data[i]))%2)
    return encrypted_data

In [4]:
def decrypt_css(key,
                encrypted_data_bin_string,
                
                prefix_for_seed_of_1st_lfsr,
                prefix_for_seed_of_2nd_lfsr,
                max_key_index_to_be_used_for_seed_of_1st_lsfr,
                max_key_index_to_be_used_for_seed_of_2nd_lsfr,
                taps):
    
    state_of_1st_lfsr = prefix_for_seed_of_1st_lfsr + key[0:max_key_index_to_be_used_for_seed_of_1st_lsfr+1]
    state_of_2nd_lfsr = prefix_for_seed_of_2nd_lfsr + key[max_key_index_to_be_used_for_seed_of_1st_lsfr + 1:
                                                         max_key_index_to_be_used_for_seed_of_2nd_lsfr + 1]
    
#     encrypted_data_byte_array = bytearray(encrypted_data,encoding)
    data = ''
    for i  in range (0,len(encrypted_data_bin_string),8):
        state_of_1st_lfsr,output_of_1st_lfsr = lfsr(seed=state_of_1st_lfsr,
                                      taps=taps,
                                      output_length=8)
        
        state_of_2nd_lfsr,output_of_2nd_lfsr = lfsr(seed=state_of_2nd_lfsr,
                                                       taps=taps,
                                                       output_length=8)
        
        
        # composite_pseudorandom_sequence = (output_of_1st_lfsr + output_of_2nd_lfsr) mod 256
        composite_pseudorandom_sequence = (int(output_of_1st_lfsr,2) + int (output_of_2nd_lfsr,2))%256
        composite_pseudorandom_sequence = '{0:08b}'.format(composite_pseudorandom_sequence)
        bin_data = encrypted_data_bin_string[i:i+8]
        data =  data + one_time_pad(key = composite_pseudorandom_sequence,
                                     data = bin_data)
    return data

In [5]:
"""

"""
def css(key,
        data,
        taps):
    if len(data)%8 !=0:
        raise ValueError("Data length must be a multiple of 8.")
    prefix_for_seed_of_1st_lfsr = '1'
    prefix_for_seed_of_2nd_lfsr = '1'
    max_key_index_to_be_used_for_seed_of_1st_lsfr = 15
    max_key_index_to_be_used_for_seed_of_2nd_lsfr = 39
    state_of_1st_lfsr = prefix_for_seed_of_1st_lfsr + key[0:max_key_index_to_be_used_for_seed_of_1st_lsfr+1]
    state_of_2nd_lfsr = prefix_for_seed_of_2nd_lfsr + key[max_key_index_to_be_used_for_seed_of_1st_lsfr + 1:
                                                         max_key_index_to_be_used_for_seed_of_2nd_lsfr + 1]
#     print("initial state of first lfsr")
#     print(state_of_1st_lfsr)
#     print("initial state of second lfsr")
#     print(state_of_2nd_lfsr)
#     data_byte_array = bytearray(data,encoding)
    encrypted_data = ''
    encrypted_data_bin = ''
    indices = []
    for k in range(0,len(data),8):
            indices.append(k)
    data_parts = [data[i:j] for i,j in zip(indices,indices[1:]+[None])]
    
    for part in data_parts:
        state_of_1st_lfsr,output_of_1st_lfsr = lfsr(seed=state_of_1st_lfsr,
                                      taps=taps,
                                      output_length=8)
        
        state_of_2nd_lfsr,output_of_2nd_lfsr = lfsr(seed=state_of_2nd_lfsr,
                                                       taps=taps,
                                                       output_length=8)
        

        # composite_pseudorandom_sequence = (output_of_1st_lfsr + output_of_2nd_lfsr) mod 256
        composite_pseudorandom_sequence = (int(output_of_1st_lfsr,2) + int (output_of_2nd_lfsr,2))%256
        composite_pseudorandom_sequence = '{0:08b}'.format(composite_pseudorandom_sequence)
        bin_data = part
        encrypted_data =  encrypted_data + one_time_pad(key = composite_pseudorandom_sequence,
                                                        data = bin_data)
    
    return encrypted_data

In [6]:
data = '01100001011100110110010001100110011001110110100001101010011010110110110001101111'
print(data)
encrypted_data = css(key = '1100001010000111000110101000000111101011',
                     data = data,
                     taps = (1,5,6,2,9))
print("Encrypted Data")
print(encrypted_data)

01100001011100110110010001100110011001110110100001101010011010110110110001101111
Encrypted Data
10110010010101000101011001011000010000100010000001110000001001011101111101001001


In [7]:
decrypted_data = decrypt_css(key = '1100001010000111000110101000000111101011',
            encrypted_data_bin_string= encrypted_data,
            prefix_for_seed_of_1st_lfsr = '1',
            prefix_for_seed_of_2nd_lfsr = '1',
            max_key_index_to_be_used_for_seed_of_1st_lsfr = 15,
            max_key_index_to_be_used_for_seed_of_2nd_lsfr = 39,
            taps=(1,5,6,2,9))
print("Decrypted Data")
print(decrypted_data)

Decrypted Data
01100001011100110110010001100110011001110110100001101010011010110110110001101111


In [17]:
def de_css(known_initial_bits_of_original_message,
           encrypted_data,
           taps):
    if  len(known_initial_bits_of_original_message)%8 != 0 or len(encrypted_data)%8 != 0:
        raise ValueError("Length of known_initial_bits_of_original_message and encrypted_data must be multiples of 8.")

    # Determine the initial bits of PRG by xoring the initial bits of encrypted data with 
    # known bits of original message.
    prg_predicted = one_time_pad(known_initial_bits_of_original_message,
                                encrypted_data[0:len(known_initial_bits_of_original_message)])

    prg_slice = prg_predicted[0:32]
    min_val_of_1st_lfsr = int('00000000000000000',2)
    max_val_of_1st_lfsr = int('11111111111111111',2)
    states_after_32_bits = []
    # Start guesses for first lfsr initial state
    
    for first_lfsr_guess in range(min_val_of_1st_lfsr,max_val_of_1st_lfsr + 1):
        guess_for_state_of_first_lfsr = '{0:017b}'.format(first_lfsr_guess)

        # Get first_lfsr_output after 32 bits
        first_lfsr_state,first_lfsr_output = lfsr(seed=guess_for_state_of_first_lfsr,
                                                  taps=(1,5,6,2),
                                                  output_length = 32)
        
        indices = []
        for k in range(0,32,8):
            indices.append(k)
        first_lfsr_output_parts = [first_lfsr_output[i:j] for i,j in zip(indices,indices[1:]+[None])]
        
        prg_slice_parts = [prg_slice[i:j] for i,j in zip(indices,indices[1:]+[None])]
        
        output_of_second_lfsr = ''
        for k in range(0,len(first_lfsr_output_parts)):
            int_output_of_second_lfsr = (int(prg_slice_parts[k],2) - 
                                         int(first_lfsr_output_parts[len(first_lfsr_output_parts)- k - 1],2))%256
            bin_output_of_second_lfsr = '{0:08b}'.format(int_output_of_second_lfsr)
            output_of_second_lfsr =  bin_output_of_second_lfsr + output_of_second_lfsr

        second_lfsr_state = output_of_second_lfsr[0:25] 
        state_discovered = ''
        bits_remaining = len(known_initial_bits_of_original_message) - 32
        for i in range(0,bits_remaining,8):
            # first_lfsr_state and output after next 8 bits
            first_lfsr_state,first_lfsr_output = lfsr(seed=first_lfsr_state,
                                                      taps=taps,
                                                      output_length=8)
            # second_lfsr_state and output after next 8 bits
            second_lfsr_state,second_lfsr_output = lfsr(seed=second_lfsr_state,
                                                      taps=taps,
                                                      output_length=8)
            next_prs = '{0:08b}'.format(((int(first_lfsr_output,2) + int(second_lfsr_output,2))%256))
            encrypted_data_slice = encrypted_data[32 + i*8:32 + (i+1)*8]
            original_data_slice = known_initial_bits_of_original_message[32 + i*8:32 + (i+1)*8]
            original_data_slice_computed = one_time_pad(key = next_prs,data = encrypted_data_slice)
            if original_data_slice_computed == original_data_slice:
                state_discovered = True
                continue
            else:
                state_discovered = False
                break
        if state_discovered:
            states_after_32_bits.append([first_lfsr_state,second_lfsr_state])
    return states_after_32_bits
        

In [19]:
states_after_32_bits = de_css(known_initial_bits_of_original_message ='0110000101110011011001000110011001100111',
       encrypted_data=encrypted_data,
       taps = (1,5,6,2,9))
print(states_after_32_bits)

[['00101000101011001', '1111110110010010100011101'], ['00101000101011001', '1111110110010010100011101'], ['00101000101011001', '1111110110010010100011101'], ['00101000101011001', '1111110110010010100011101'], ['00101000101011001', '1111110110010010100011101'], ['00101000101011001', '1111110110010010100011101'], ['00101000101011001', '1111110110010010100011101'], ['00101000101011001', '1111110110010010100011101'], ['00101000101011001', '1111110110010010100011101'], ['00101000101011001', '1111110110010010100011101'], ['00101000101011001', '1111110110010010100011101'], ['00101000101011001', '1111110110010010100011101'], ['00101000101011001', '1111110110010010100011101'], ['00101000101011001', '1111110110010010100011101'], ['00101000101011001', '1111110110010010100011101'], ['00101000101011001', '1111110110010010100011101'], ['00101000101011001', '1111110110010010100011101'], ['00101000101011001', '1111110110010010100011101'], ['00101000101011001', '1111110110010010100011101'], ['001010001